In [ ]:
import os
import sys
import json
import pandas as pd
import numpy as np 

from CoolProp.CoolProp import PropsSI

# shortcuts
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

sys.path.append(norm(join(cwd, '..', '..', '..', 'glhe')))
sys.path.append(norm(join(cwd, '..', '..', '..', 'standalone')))

from standalone.run_g_function import RunGFunctions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
constant_trt_path = '1-s2.0-S0375650511000022-mmc1.xls'

In [ ]:
df_const = pd.read_excel(constant_trt_path, sheet_name='TRT degC', skiprows=7, names=['Minutes', 
                                                                                      'Temp Water Supply [C]',
                                                                                      'Temp Water Return [C]',
                                                                                      'Temp Avg [C]',
                                                                                      'Flow Rate [lps]',
                                                                                      'Electric Power [W]',
                                                                                      'Temp Diff [C]',
                                                                                      'Calorimetric Power [W]'])

# df_const['Time'] = pd.to_timedelta(pd.to_datetime(df_const['Minutes'], unit='m'))
df_const['Date/Time'] = pd.to_datetime(df_const['Minutes'], unit='m')
df_const.drop('Minutes', axis=1, inplace=True)
df_const.set_index('Date/Time', inplace=True)
df_const.apply(pd.to_numeric)
df_const.head()

In [ ]:
df_const.plot(y=['Temp Water Supply [C]', 'Temp Water Return [C]'])
df_const.plot(y=['Electric Power [W]', 'Calorimetric Power [W]'])

In [ ]:
def C_to_K(inputs):
    return inputs + 273.15

In [ ]:
df_const['Mass Flow Rate [kg/s]'] = df_const['Flow Rate [lps]'] * PropsSI('D', 'T', C_to_K(df_const['Temp Avg [C]'].values), 'P', 101325, 'Water') / 1000

In [ ]:
timestep = 2 # minutes
df_const = df_const.resample('{}T'.format(timestep)).mean()
df_const = df_const.interpolate(method='linear')
df_const.head()

In [ ]:
df_const.plot(y=['Temp Water Supply [C]', 'Temp Water Return [C]'])
df_const.plot(y=['Electric Power [W]', 'Calorimetric Power [W]'])
df_const.plot(y=['Flow Rate [lps]', 'Mass Flow Rate [kg/s]'])

In [ ]:
df_const.to_csv('in.csv', columns=['Calorimetric Power [W]', 'Mass Flow Rate [kg/s]'])

In [ ]:
runtime = 50 * 3600

in_file = join(cwd, 'Sandbox_input.json')
with open(in_file, 'r') as f:
    json_blob = f.read()
in_json = json.loads(json_blob)

in_json['simulation']['runtime'] = runtime
in_json['simulation']['time-steps per hour'] = int(60 / timestep)

in_json['load-aggregation']['type'] = 'dynamic'

in_json['load-aggregation']['dynamic'] = {}
in_json['load-aggregation']['dynamic']['expansion rate'] = np.exp(0.2)
in_json['load-aggregation']['dynamic']['runtime'] = in_json['simulation']['runtime']

in_json['simulation']['output-path'] = join(cwd, 'out.csv')

with open(join(cwd, 'in.json'), 'w') as f:
    f.write(json.dumps(in_json, sort_keys=True, indent=2))
    
in_file = join(cwd, 'in.json')

In [ ]:
RunGFunctions(in_file).simulate()

In [ ]:
df_output = pd.read_csv("out.csv", index_col='Simulation Time')

In [ ]:
df_output['Time'] = pd.to_datetime(df_output.index, unit='s')
df_output.set_index('Time', inplace=True)
df_output.head(3)

In [ ]:
start_pt = 0
end_pt = 200

in_model = df_output['GLHE Inlet Temperature [C]'].iloc[start_pt:end_pt]
out_model = df_output['GLHE Outlet Temperature [C]'].iloc[start_pt:end_pt]
plt.plot(in_model, label='In (Model)', linestyle='-.')
plt.plot(out_model, label='Out (Model)', linestyle='-.')

in_exp = df_const['Temp Water Supply [C]'].iloc[start_pt:end_pt]
out_exp = df_const['Temp Water Return [C]'].iloc[start_pt:end_pt]
plt.plot(in_exp, label='In (Exp)', marker='x')
plt.plot(out_exp, label='Out (Exp)', marker='x')

plt.legend()
plt.show()